# Dependencies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random

# Data Creation

In [2]:
def generate_sentence(max_len=20, pos=True):
    if pos:
        n = np.random.randint(np.ceil(max_len / 3))
        sentence = n*"a" + n*"b" + n*"c"
        return sentence, len(sentence)
    else:
        n_0 = np.random.randint(0, max_len)
        n_1 = np.random.randint(0, max_len - n_0 + 1)
        n_2 = np.random.randint(0, max_len - n_0 - n_1 + 1)
        sentence = n_0 * "a" + n_1 * "b" + n_2 * "c"
        return sentence, len(sentence)

def create_data(size=10000, balance=0.5):
    data = []
    sentence_lengths = []

    for i in range(int(size*balance)):
        sentence, sentence_length = generate_sentence(pos=True)
        data.append((sentence, 1))
        sentence_lengths.append(sentence_length)
    for i in range(int((size - (size*balance)))):
        sentence, sentence_length = generate_sentence(pos=False)
        data.append((sentence, 0))
        sentence_lengths.append(sentence_length)
    
    random.shuffle(data)
    average_length = sum(sentence_lengths) / len(sentence_lengths)
    
    return data, average_length

data, avg_sent_length = create_data()
print(f"Data:\n{data}")
print(f"Average Sentence Length:\n{avg_sent_length}")


Data:
[('aaaaaaaaaaaaaaaabcc', 0), ('abc', 1), ('aaaaabbccccccccc', 0), ('aaabbbccc', 1), ('aaaaaaaabbbbbcc', 0), ('aaaaaabbbbbbcccccc', 1), ('aaabbbbbbbbbbbbbbccc', 0), ('aaaaaaaabbbbbbbbbb', 0), ('aaaabbbbcccc', 1), ('aaaaaaaaaaaaaaaaaabb', 0), ('aaaaaaaaaaabbbbbb', 0), ('aaaaaaaaaaaaaaabbbc', 0), ('aaabbbccc', 1), ('', 1), ('aaaaaabbbbbbcccccc', 1), ('abc', 1), ('aaaaaabbbbbbcccccc', 1), ('aaaaabbbbbccccc', 1), ('aaaabbbbcccc', 1), ('aaaabbbbcccc', 1), ('aaaaabbbbbbbbbbbb', 0), ('aaabbbccc', 1), ('', 1), ('aaabbbccc', 1), ('aaabbbccc', 1), ('aaaaaaaaaaaaaaaac', 0), ('aaaaaaaaaaaaaaaaaccc', 0), ('abc', 1), ('aaaabbbbbbbbbbbbbbb', 0), ('aaaabbbbcccc', 1), ('aabbcc', 1), ('aaaaaaaaaaaaaaaccc', 0), ('aaaaabbbbbbbbcccc', 0), ('aaaabbbbcccc', 1), ('aaaaaaaaaaaaccccc', 0), ('aaaaabbbbcccccccccc', 0), ('', 1), ('aaaabbbbcccc', 1), ('aaaaaabbbbbbcccccc', 1), ('aaaaaaaaabbbbcc', 0), ('abc', 1), ('aaaaabbbbbccccc', 1), ('abc', 1), ('aaaaaaaaabbbbbbbcccc', 0), ('aaaaaabbbbbbcccccc', 1), ('aaaaa

# Part B: Recurrent Neural Networks

In [18]:
# Dependencies
import torch
import torch.nn as nn
import torch.optim as optim

In [36]:
# Encoding data
char_to_index = {'a':0, 'b':1, 'c':2}
index_to_char = {v: k for k, v in char_to_index.items()}

def creat_tensors():
    X = []
    y = []
    ml = 0

    for sent, label in data:
        X.append([char_to_index[char] for char in sent])
        y.append(label)
        ml = max(ml, len([char_to_index[char] for char in sent]))

    X = [sequence + [0] * (ml - len(sequence)) for sequence in X]

    X = torch.tensor(X, dtype=torch.long)
    y = torch.tensor(y, dtype=torch.float32)

    return X, y

X_train, y_train = creat_tensors()

### RNN

In [4]:
# Set device
if torch.cuda.is_available():
    device = 'cuda:0'
elif torch.backends.mps.is_available():
    device = 'mps:0'
else:
    device = 'cpu'

print('GPU State:', device)

GPU State: mps:0


In [ ]:
class RNN(nn.Module):
    def __init__
    pass

### LSTM

In [11]:
pass